In [1]:
import numpy as np
import pandas as pd

import pickle

In [2]:
df = pd.read_pickle('../Data/01_sf_recommender_pca')

In [3]:
df.head()

,company_name,job_title,listed_items,posting_url,0,1,2,3,4,5,...,290,291,292,293,294,295,296,297,298,299
0,Gap Inc. Corporate,"Software Engineer, Price Execution",write build product according business conduct...,https://www.indeed.com/rc/clk?jk=77d524a7cf198...,-0.018077,0.014864,0.117981,0.040068,-0.062345,0.038817,...,-0.014063,-0.019327,0.013030,0.008991,0.003695,-0.012031,0.005944,0.020383,0.009977,-0.004942
1,WrkShp,Business Analyst,closely product assist investigation deep dive...,https://www.indeed.com/company/WrkShp/jobs/Bus...,-0.049036,0.180958,-0.111539,0.089243,-0.110111,-0.009823,...,0.020357,0.018804,-0.018694,0.023452,-0.002981,0.039731,0.034997,-0.020702,-0.001777,-0.041647
2,Ceres Imaging,Image Processing: GIS / Remote Sensing Analyst,proficiency gi e g arcgis envi processing prod...,https://www.indeed.com/rc/clk?jk=8f702cd563785...,0.068104,-0.108209,-0.024293,0.013824,0.080689,-0.107695,...,0.014435,-0.006349,-0.022195,0.007054,-0.001135,-0.001025,-0.009957,-0.005164,0.004571,-0.005716
3,Deloitte,"Analyst, Strategy and Research",effectively interpret client request use tacti...,https://www.indeed.com/rc/clk?jk=8a288a5c5a09d...,-0.076938,-0.030794,-0.033607,0.040571,0.034268,-0.016858,...,0.002196,0.015415,0.031909,0.051545,0.008766,-0.005510,0.010462,0.027878,-0.020657,-0.001136
4,Turing Video,Computer Vision Software Engineer,maintain existing implement algorithm necessar...,https://www.indeed.com/rc/clk?jk=fcf308f2fee2a...,0.144809,0.012746,0.025459,-0.029213,0.056539,0.029222,...,-0.057134,0.033395,-0.033605,0.011401,-0.020123,0.008876,-0.033616,-0.039308,0.006560,0.051977


Make recommendation from existing job posting

# Rough draft of recommender functions for app.py script

## Testing making a recommendation based on an existing job posting

``` python
def make_recommendation(df, job_index,n_recommendations=5):
    n_recommendations += 1
    df['similarity'] = df.iloc[:,4:].T.apply(lambda x: np.dot(x,df.iloc[job_index,4:]))
    largest_indeces = df.similarity.nlargest(n_recommendations).index
    largest_indeces = largest_indeces[1:]
    recommendations = df.iloc[largest_indeces].posting_url
    original_posting = df.iloc[job_index].posting_url
    return recommendations, original_posting
```

``` python 
recommendations, original_posting = make_recommendation(df, 2479,n_recommendations=5)
original_posting
recommendations.values
df.iloc[2479].posting_url
df.shape
```

## Creating Pipeline for reading text, vecotrizing it, and creating a recommendation

## Turn Text Description into a Vector

In [4]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import copy

In [5]:
job_desc = 'Research and experiment with different machine learning algorithms and techniques for item classification, product recognition and matching, attribute extraction. \
            Conduct design and code reviews. \
            Productionize the developed Machine Learning solutions. \
            Work with engineers to make sure the engines scale well on high volumes of data. \
            MS/PhD in Computer Science or a related field. \
            3+ years of industry experience in data science and machine learning preferred. \
            Expertise in Machine Learning. \
            Strong CS fundamentals, such as algorithms and data structures. \
            Expertise in Python. \
            Proficiency with relational databases such as MySQL. \
            Experience with cloud computing stacks such as Amazon Web Services preferred. \
            Excellent written and verbal communication skills. \
            Enthusiasm for working hard and having fun in a dynamic environment.'
        


Research and experiment with different machine learning algorithms and techniques for item classification, product recognition and matching, attribute extraction
Conduct design and code reviews
Productionize the developed Machine Learning solutions
Work with engineers to make sure the engines scale well on high volumes of data
MS/PhD in Computer Science or a related field
3+ years of industry experience in data science and machine learning preferred
Expertise in Machine Learning
Strong CS fundamentals, such as algorithms and data structures
Expertise in Python
Proficiency with relational databases such as MySQL
Experience with cloud computing stacks such as Amazon Web Services preferred
Excellent written and verbal communication skills
Enthusiasm for working hard and having fun in a dynamic environment

In [6]:
def get_input():
    "Gets input from the user."
    return input()

In [7]:
job_desc = get_input()

 Research and experiment with different machine learning algorithms and techniques for item classification, product recognition and matching, attribute extraction Conduct design and code reviews Productionize the developed Machine Learning solutions Work with engineers to make sure the engines scale well on high volumes of data MS/PhD in Computer Science or a related field 3+ years of industry experience in data science and machine learning preferred Expertise in Machine Learning Strong CS fundamentals, such as algorithms and data structures Expertise in Python Proficiency with relational databases such as MySQL Experience with cloud computing stacks such as Amazon Web Services preferred Excellent written and verbal communication skills Enthusiasm for working hard and having fun in a dynamic environment


In [8]:
def format_text(text):
    "Reads a single string. Removes punctuation and makes text lowercase. Removes stopwords"
    
    text = re.sub('[\W_]+', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text.lower())
    
    stop = set(stopwords.words('english'))
    text = [item for item in text.split() if item not in stop]
    
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(token) for token in text]
    
    stop2 = ['data', 'code', 'team', 'work']
    text = [item for item in text if item not in stop2]
    
    return text

In [9]:
job_desc = format_text(job_desc)

In [10]:
def create_ngrams(text):
    
    doc_ngrams = []
    
    for i, j in zip(text, text[1:]):
        bigram = '_'.join([i, j])
        doc_ngrams.append(bigram)
        
    for i, j, k in zip(text, text[1:], text[2:]):
        trigram = '_'.join([i, j, k])
        doc_ngrams.append(trigram)
        
    for i, j, k, l in zip(text, text[1:],
                          text[2:], text[3:]):
        quadgram = '_'.join([i, j, k, l])
        doc_ngrams.append(quadgram)
        
    pickle_in = open('../Tools_and_models/top_ngrams','rb')
    top_ngrams = pickle.load(pickle_in)
    pickle_in.close()
        
    top_ngrams = top_ngrams.split()
    
    for doc_ngram in doc_ngrams:
        for top_ngram in top_ngrams:
            if doc_ngram == top_ngram:
                text.append(doc_ngram)
            else:
                pass
    return ' '.join(text)
    

In [11]:
job_desc = create_ngrams(job_desc)

In [12]:
def vectorize(text):
    pickle_in = open('../Tools_and_models/tf_idf_vectorizer','rb')
    tf_idf_vectorizer = pickle.load(pickle_in)
    pickle_in.close()
    
    pickle_in = open('../Tools_and_models/tf_idf_model','rb')
    tf_idf = pickle.load(pickle_in)
    pickle_in.close()
    
    text_vector_array = tf_idf_vectorizer.transform([text]).toarray()
    
    text_vector = pd.DataFrame(text_vector_array,columns=tf_idf.get_feature_names())
    
    return text_vector
    

In [13]:
text_vector = vectorize(job_desc)
text_vector

,aa,aaa,aaai,aac,aad,aami,aapc,aaproot_cause,aashto,aav,...,zoura,zpn,zuckerberg,zurb,zvs,zweigwhite,zymergen,zymo,zynga,ºc
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
def pcaitize(text_vector):

    pickle_in = open('../Tools_and_models/pca_tool','rb')
    pca = pickle.load(pickle_in)
    pickle_in.close()
    
    text_pca = pca.transform(text_vector)
    
    return pd.DataFrame(text_pca)

In [15]:
text_pca = pcaitize(text_vector)
text_pca

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.280428,0.102534,-0.093753,-0.161354,0.021741,0.134529,0.003344,-0.012962,-0.009692,-0.022092,...,0.021529,-0.009893,-0.001702,0.019538,-0.019995,-0.043382,-0.014717,0.029268,0.025065,-0.014068


## Making a search based on our text_pca

In [26]:
def recommender(df, text_pca,n_recommendations=5):
    recos = pd.DataFrame(df.iloc[:,4:].T.apply(lambda x: np.dot(x,text_pca.iloc[0,:])))
    largest_indeces = recos[0].nlargest(n_recommendations).index
    urls = df.iloc[largest_indeces].posting_url.values
    companies = df.iloc[largest_indeces].company_name.values
    titles = df.iloc[largest_indeces].job_title.values
    return urls, companies, titles

In [27]:
urls, companies, titles = recommender(df, text_pca,5)
print(urls)
print(companies)
print(titles)

['https://www.indeed.com/rc/clk?jk=3c1a65c54c18cdad&fccid=99f81b9c51cc1932&vjs=3'
 'https://www.indeed.com/rc/clk?jk=9200b3748d19991f&fccid=2fbf50740df63658&vjs=3'
 'https://www.indeed.com/rc/clk?jk=a186c0b47d17a8a1&fccid=42e50b7b66ee4856&vjs=3'
 'https://www.indeed.com/rc/clk?jk=e150c787ca1b1cc8&fccid=90344e51e9e06acc&vjs=3'
 'https://www.indeed.com/rc/clk?jk=a74312c657a2db0b&fccid=98eb9367230d83da&vjs=3']
['Rakuten Intelligence' 'Clusterone' 'LeapYear' '' 'C3']
['Data Scientist / Machine Learning Engineer - Classification Science'
 'Research Scientist/Engineer - SF/Bay Area'
 'Software Engineer - Machine Learning R&D'
 'Senior Applied Scientist - Machine Learning' 'Data Scientist (Federal)']


In [18]:
df.head()

,company_name,job_title,listed_items,posting_url,0,1,2,3,4,5,...,290,291,292,293,294,295,296,297,298,299
0,Gap Inc. Corporate,"Software Engineer, Price Execution",write build product according business conduct...,https://www.indeed.com/rc/clk?jk=77d524a7cf198...,-0.018077,0.014864,0.117981,0.040068,-0.062345,0.038817,...,-0.014063,-0.019327,0.013030,0.008991,0.003695,-0.012031,0.005944,0.020383,0.009977,-0.004942
1,WrkShp,Business Analyst,closely product assist investigation deep dive...,https://www.indeed.com/company/WrkShp/jobs/Bus...,-0.049036,0.180958,-0.111539,0.089243,-0.110111,-0.009823,...,0.020357,0.018804,-0.018694,0.023452,-0.002981,0.039731,0.034997,-0.020702,-0.001777,-0.041647
2,Ceres Imaging,Image Processing: GIS / Remote Sensing Analyst,proficiency gi e g arcgis envi processing prod...,https://www.indeed.com/rc/clk?jk=8f702cd563785...,0.068104,-0.108209,-0.024293,0.013824,0.080689,-0.107695,...,0.014435,-0.006349,-0.022195,0.007054,-0.001135,-0.001025,-0.009957,-0.005164,0.004571,-0.005716
3,Deloitte,"Analyst, Strategy and Research",effectively interpret client request use tacti...,https://www.indeed.com/rc/clk?jk=8a288a5c5a09d...,-0.076938,-0.030794,-0.033607,0.040571,0.034268,-0.016858,...,0.002196,0.015415,0.031909,0.051545,0.008766,-0.005510,0.010462,0.027878,-0.020657,-0.001136
4,Turing Video,Computer Vision Software Engineer,maintain existing implement algorithm necessar...,https://www.indeed.com/rc/clk?jk=fcf308f2fee2a...,0.144809,0.012746,0.025459,-0.029213,0.056539,0.029222,...,-0.057134,0.033395,-0.033605,0.011401,-0.020123,0.008876,-0.033616,-0.039308,0.006560,0.051977


In [19]:
def make_recommendations(df, n_recommendations=5):
    text = get_input()
    formatted_text = format_text(text)
    ngramed_text = create_ngrams(formatted_text)
    text_vector = vectorize(ngramed_text)
    text_pca = pcaitize(text_vector)
    recommendations = recommender(df, text_pca,n_recommendations=5)
    return recommendations

In [20]:
make_recommendations(df, n_recommendations=5)

 Research and experiment with different machine learning algorithms and techniques for item classification, product recognition and matching, attribute extraction Conduct design and code reviews Productionize the developed Machine Learning solutions Work with engineers to make sure the engines scale well on high volumes of data MS/PhD in Computer Science or a related field 3+ years of industry experience in data science and machine learning preferred Expertise in Machine Learning Strong CS fundamentals, such as algorithms and data structures Expertise in Python Proficiency with relational databases such as MySQL Experience with cloud computing stacks such as Amazon Web Services preferred Excellent written and verbal communication skills Enthusiasm for working hard and having fun in a dynamic environment


2116    https://www.indeed.com/rc/clk?jk=3c1a65c54c18c...
1935    https://www.indeed.com/rc/clk?jk=9200b3748d199...
1671    https://www.indeed.com/rc/clk?jk=a186c0b47d17a...
675     https://www.indeed.com/rc/clk?jk=e150c787ca1b1...
26      https://www.indeed.com/rc/clk?jk=a74312c657a2d...
Name: posting_url, dtype: object